In [2]:
load("GeneralizedShallowWaterVariables.sage")
load('../Sage/VectorMatrixVariables.sage')
num_moments = 1
set_generalized_shallow_water_variables(num_moments)
num_basis_cpts = 2 + num_moments

s = var('s', domain="positive")
# Local Lax Friedrichs
c = -s
d = s
I = matrix.identity(num_basis_cpts)
B = block_matrix([[-Q_p, I],[-c*d*I, (c + d)*I]])
print(B)

[  0   0   0|  1   0   0]
[  0   0   0|  0   1   0]
[  0   0  -u|  0   0   1]
[-----------+-----------]
[s^2   0   0|  0   0   0]
[  0 s^2   0|  0   0   0]
[  0   0 s^2|  0   0   0]


In [3]:
eig = B.eigenvectors_right()
eigenvalues = flatten([[e[0].simplify_full() for i in range(e[2])] for e in eig])
eigenvectors = flatten([[ev.simplify_full() for ev in e[1]] for e in eig])
R = matrix(eigenvectors).T
L = R.inverse().simplify_full()
print(eigenvalues)
print(R)

[-1/2*u - 1/2*sqrt(4*s^2 + u^2), -1/2*u + 1/2*sqrt(4*s^2 + u^2), -s, -s, s, s]
[                            0                             0                             1                             0                             1                             0]
[                            0                             0                             0                             1                             0                             1]
[                            1                             1                             0                             0                             0                             0]
[                            0                             0                            -s                             0                             s                             0]
[                            0                             0                             0                            -s                             0                             s]
[1/2*u - 1/

In [4]:
Q_r = get_vector_variable("Q_r", num_basis_cpts)
Q_l = get_vector_variable("Q_l", num_basis_cpts)
f_Q_r = get_vector_variable("f_Q_r", num_basis_cpts)
f_Q_l = get_vector_variable("f_Q_l", num_basis_cpts)
Q_diff = Q_r - Q_l
f_diff = f_Q_r - f_Q_l
v = vector(Q_diff.list() + f_diff.list())
alphas = (L*v).simplify_full()
print(alphas)

(-1/2*(4*(Q_l_2 - Q_r_2)*s^2 + (Q_l_2 - Q_r_2)*u^2 + sqrt(4*s^2 + u^2)*((Q_l_2 - Q_r_2)*u - 2*f_Q_l_2 + 2*f_Q_r_2))/(4*s^2 + u^2), -1/2*(4*(Q_l_2 - Q_r_2)*s^2 + (Q_l_2 - Q_r_2)*u^2 - sqrt(4*s^2 + u^2)*((Q_l_2 - Q_r_2)*u - 2*f_Q_l_2 + 2*f_Q_r_2))/(4*s^2 + u^2), -1/2*((Q_l_0 - Q_r_0)*s - f_Q_l_0 + f_Q_r_0)/s, -1/2*((Q_l_1 - Q_r_1)*s - f_Q_l_1 + f_Q_r_1)/s, -1/2*((Q_l_0 - Q_r_0)*s + f_Q_l_0 - f_Q_r_0)/s, -1/2*((Q_l_1 - Q_r_1)*s + f_Q_l_1 - f_Q_r_1)/s)


In [99]:
(R*alphas - v).simplify_full()

(0, 0, 0, 0, 0, 0)

In [5]:
# wave propogation
Am_deltaQ = (eigenvalues[0]*alphas[0]*eigenvectors[0][:num_basis_cpts] + eigenvalues[2]*alphas[2]*eigenvectors[2][:num_basis_cpts] + eigenvalues[3]*alphas[3]*eigenvectors[3][:num_basis_cpts]).simplify_full()
#sum([(min(eigenvalues[i], 0)*alphas[i]*eigenvectors[i][:num_basis_cpts]).simplify_full() for i in range(2*num_basis_cpts)])
print(Am_deltaQ)
Ap_deltaQ = (eigenvalues[1]*alphas[1]*eigenvectors[1][:num_basis_cpts] + eigenvalues[4]*alphas[4]*eigenvectors[4][:num_basis_cpts] + eigenvalues[5]*alphas[5]*eigenvectors[5][:num_basis_cpts]).simplify_full()
#sum([(max(eigenvalues[i], 0)*alphas[i]*eigenvectors[i][:num_basis_cpts]).simplify_full() for i in range(2*num_basis_cpts)])
print(Ap_deltaQ)

(1/2*(Q_l_0 - Q_r_0)*s - 1/2*f_Q_l_0 + 1/2*f_Q_r_0, 1/2*(Q_l_1 - Q_r_1)*s - 1/2*f_Q_l_1 + 1/2*f_Q_r_1, 1/2*(4*(Q_l_2 - Q_r_2)*s^2*u + (Q_l_2 - Q_r_2)*u^3 - 4*(f_Q_l_2 - f_Q_r_2)*s^2 - (f_Q_l_2 - f_Q_r_2)*u^2 + (2*(Q_l_2 - Q_r_2)*s^2 + (Q_l_2 - Q_r_2)*u^2 - (f_Q_l_2 - f_Q_r_2)*u)*sqrt(4*s^2 + u^2))/(4*s^2 + u^2))
(-1/2*(Q_l_0 - Q_r_0)*s - 1/2*f_Q_l_0 + 1/2*f_Q_r_0, -1/2*(Q_l_1 - Q_r_1)*s - 1/2*f_Q_l_1 + 1/2*f_Q_r_1, 1/2*(4*(Q_l_2 - Q_r_2)*s^2*u + (Q_l_2 - Q_r_2)*u^3 - 4*(f_Q_l_2 - f_Q_r_2)*s^2 - (f_Q_l_2 - f_Q_r_2)*u^2 - (2*(Q_l_2 - Q_r_2)*s^2 + (Q_l_2 - Q_r_2)*u^2 - (f_Q_l_2 - f_Q_r_2)*u)*sqrt(4*s^2 + u^2))/(4*s^2 + u^2))


In [6]:
# numerical flux
F_l = (f_Q_l + Am_deltaQ).simplify_full()
print(F_l)
F_r = (f_Q_r - Ap_deltaQ).simplify_full()
print(F_r)

(1/2*(Q_l_0 - Q_r_0)*s + 1/2*f_Q_l_0 + 1/2*f_Q_r_0, 1/2*(Q_l_1 - Q_r_1)*s + 1/2*f_Q_l_1 + 1/2*f_Q_r_1, 1/2*(4*(Q_l_2 - Q_r_2)*s^2*u + (Q_l_2 - Q_r_2)*u^3 + 4*(f_Q_l_2 + f_Q_r_2)*s^2 + (f_Q_l_2 + f_Q_r_2)*u^2 + (2*(Q_l_2 - Q_r_2)*s^2 + (Q_l_2 - Q_r_2)*u^2 - (f_Q_l_2 - f_Q_r_2)*u)*sqrt(4*s^2 + u^2))/(4*s^2 + u^2))
(1/2*(Q_l_0 - Q_r_0)*s + 1/2*f_Q_l_0 + 1/2*f_Q_r_0, 1/2*(Q_l_1 - Q_r_1)*s + 1/2*f_Q_l_1 + 1/2*f_Q_r_1, -1/2*(4*(Q_l_2 - Q_r_2)*s^2*u + (Q_l_2 - Q_r_2)*u^3 - 4*(f_Q_l_2 + f_Q_r_2)*s^2 - (f_Q_l_2 + f_Q_r_2)*u^2 - (2*(Q_l_2 - Q_r_2)*s^2 + (Q_l_2 - Q_r_2)*u^2 - (f_Q_l_2 - f_Q_r_2)*u)*sqrt(4*s^2 + u^2))/(4*s^2 + u^2))


In [110]:
(((f_Q_r_2*q_1 + (Q_l_2 - Q_r_2)*q_2)/q_1) - ((f_Q_r_2 + (Q_l_2 - Q_r_2)*q_2/q_1))).simplify_full()

0

In [30]:
(F_l[2] - F_r[2]).simplify_full()

(Q_l_2 - Q_r_2)*u

-1/2*(q_2 + sqrt(4*q_1^2*s^2 + q_2^2))/q_1

In [23]:
(alphas[0]*eigenvalues[0]).simplify_full()

1/2*(4*(Q_l_2 - Q_r_2)*s^2*u + (Q_l_2 - Q_r_2)*u^3 - 4*(f_Q_l_2 - f_Q_r_2)*s^2 - (f_Q_l_2 - f_Q_r_2)*u^2 + (2*(Q_l_2 - Q_r_2)*s^2 + (Q_l_2 - Q_r_2)*u^2 - (f_Q_l_2 - f_Q_r_2)*u)*sqrt(4*s^2 + u^2))/(4*s^2 + u^2)

In [18]:
alphas[3]

-1/2*((Q_l_1 - Q_r_1)*s - f_Q_l_1 + f_Q_r_1)/s

In [19]:
eigenvalues[0]

-1/2*u - 1/2*sqrt(4*s^2 + u^2)

In [10]:
eigenvectors[0]

(0, 0, 1, 0, 0, 1/2*u - 1/2*sqrt(4*s^2 + u^2))

In [14]:
eigenvectors[2]

(1, 0, 0, -s, 0, 0)

In [36]:
(F_r[2]+F_l[2]).simplify_full()

(4*(f_Q_l_2 + f_Q_r_2)*s^2 + (f_Q_l_2 + f_Q_r_2)*u^2 + (2*(Q_l_2 - Q_r_2)*s^2 + (Q_l_2 - Q_r_2)*u^2 - (f_Q_l_2 - f_Q_r_2)*u)*sqrt(4*s^2 + u^2))/(4*s^2 + u^2)

In [25]:
f_Q_l[2]

f_Q_l_2